# lstmgan Model

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class LSTMGAN(object):
    """ LSTM-GAN implementation in tensorflow.

        Args:
            seq_size (int): Max size of sentence.
            vocab_size (int): Size of words vocabulary.
            first_input (int): Code of first word in generator.
            hidden_size_gen (int, optional): Size of hidden layer
                in generator.
            hidden_size_disc (int, optional): Size of hidden layer
                in discriminator.
            input_noise_size (int, optional): Size of input noise
                in generator input.
            batch_size (int, optional): Batch size.
            dropout (float, optional): dropout rate.
            lr (float, optional): learning rate in Adam optimizer.
            grad_cap (float, optional): gradient cap value.

    """
    def __init__(self, seq_size, vocab_size, first_input,
                 hidden_size_gen = 512, hidden_size_disc = 512,
                 input_noise_size = 32, batch_size = 128, dropout = 0.2,
                 lr = 1e-4, grad_cap = 1.):

        self.seq_size = seq_size
        self.vocab_size = vocab_size
        self.hidden_size_gen = hidden_size_gen
        self.hidden_size_disc = hidden_size_disc
        self.input_noise_size = input_noise_size
        self.batch_size = batch_size
        self.first_input = first_input
        self.keep_prob = 1 - dropout
        self.lr = lr
        self.grad_cap = grad_cap

        self.build_model()
        self.build_trainers()

    def train_gen_on_batch(self, session, batch):
        """Train generator on given `batch` in current `session`"""
        feed = {
            self.input_noise: batch
        }
        ret_values = [self.gen_cost, self.gen_train]
        cost, _ = session.run(ret_values, feed_dict = feed)
        return cost

    def train_disc_on_batch(self, session, noise_batch, real_batch):
        """Train discriminator on given `noise_batch` and `real_batch`
        in current `session`

        """
        feed = {
            self.input_noise: noise_batch,
            self.real_sent : real_batch
        }
        ret_values = [self.disc_cost, self.disc_train]
        cost, _ = session.run(ret_values, feed_dict = feed)
        return cost

    def generate_sent(self, session, noise):
        """Generate one sentence in current `session` with given `noise`"""
        feed_dict = {self.input_noise_one_sent: [noise]}
        generated = session.run(self.sent_generator, feed_dict = feed_dict)
        return np.argmax(generated[0], axis=1)

    def build_model(self):
        batch_size, input_noise_size, seq_size, vocab_size = \
            self.batch_size, self.input_noise_size, \
            self.seq_size, self.vocab_size

        embedding = tf.diag(np.ones((vocab_size, ), dtype=np.float32))
        self.embedding = embedding

        input_noise = tf.placeholder(tf.float32, [batch_size, input_noise_size])
        input_noise_one_sent = tf.placeholder(tf.float32, [1, input_noise_size])
        self.input_noise = input_noise
        self.input_noise_one_sent = input_noise_one_sent

        real_sent = tf.placeholder(tf.int32, [batch_size, seq_size])
        input_sentence = tf.nn.embedding_lookup(embedding, real_sent)
        self.real_sent = real_sent

        _, gen_vars = self.build_generator(input_noise, is_train = True)
        generated_sent, _ = self.build_generator(input_noise, reuse = True)
        sent_generator, _ = self.build_generator(input_noise_one_sent, reuse = True)
        self.gen_vars = gen_vars
        self.generated_sent = generated_sent
        self.sent_generator = sent_generator

        _, disc_vars = self.build_discriminator(input_sentence, is_train = True)
        desc_decision_fake, _ = self.build_discriminator(generated_sent, reuse = True)
        disc_decision_real, _ = self.build_discriminator(input_sentence, reuse = True)
        self.disc_vars = disc_vars
        self.desc_decision_fake = desc_decision_fake
        self.disc_decision_real = disc_decision_real

        self.gen_cost = 1. - desc_decision_fake
        self.disc_cost = 1. - disc_decision_real*(1. - desc_decision_fake)


    def build_trainers(self):
        cap, lr, disc_cost, disc_vars, gen_cost, gen_vars = \
            self.grad_cap, self.lr, \
            self.disc_cost, self.disc_vars, \
            self.gen_cost, self.gen_vars

        optimizer_disc = tf.train.AdamOptimizer(lr)
        gvs = optimizer_disc.compute_gradients(disc_cost, disc_vars)
        capped_grads_and_vars = [(tf.clip_by_value(grad, -cap, cap), var) \
                                 for grad, var in gvs]
        optimizer_disc.apply_gradients(capped_grads_and_vars)

        optimizer_gen = tf.train.AdamOptimizer(lr)
        gvs = optimizer_gen.compute_gradients(gen_cost, gen_vars)
        capped_grads_and_vars = [(tf.clip_by_value(grad, -cap, cap), var) \
                                 for grad, var in gvs]
        optimizer_gen.apply_gradients(capped_grads_and_vars)

        self.disc_train = optimizer_disc.minimize(disc_cost)
        self.gen_train = optimizer_gen.minimize(gen_cost)

    def build_generator(self, input_, reuse = False, is_train = False):
        vocab_size, hidden_size_gen, input_noise_size, seq_size, keep_prob = \
            self.vocab_size, self.hidden_size_gen, \
            self.input_noise_size, self.seq_size, \
            self.keep_prob
        embedding, first_input = self.embedding, self.first_input

        with tf.variable_scope('generator_model', reuse = reuse):
            input_noise_w = tf.get_variable(
                "input_noise_w",
                [input_noise_size, hidden_size_gen],
                initializer=tf.random_normal_initializer(0, stddev=1/np.sqrt(vocab_size))
            )
            input_noise_b = tf.get_variable(
                "input_noise_b",
                [hidden_size_gen],
                initializer=tf.constant_initializer(1e-4)
            )
            
            first_hidden_state = tf.nn.relu(tf.matmul(input_, input_noise_w) + input_noise_b)

            cell = tf.contrib.rnn.GRUCell(hidden_size_gen)
            if is_train:
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)

            input_w = tf.get_variable(
                "input_w",
                [vocab_size, hidden_size_gen],
                initializer=tf.random_normal_initializer(0, stddev=1/np.sqrt(vocab_size))
            )
            input_b = tf.get_variable(
                "input_b",
                [hidden_size_gen],
                initializer=tf.constant_initializer(1e-4)
            )

            softmax_w = tf.get_variable(
                "softmax_w",
                [hidden_size_gen, vocab_size],
                initializer = tf.random_normal_initializer(0, stddev=1/np.sqrt(hidden_size_gen))
            )
            softmax_b = tf.get_variable(
                "softmax_b",
                [vocab_size],
                initializer=tf.constant_initializer(1e-4)
            )

            state = first_hidden_state

            labels = tf.fill([tf.shape(input_)[0], 1], tf.cast(first_input, tf.int32))
            input_ = tf.nn.embedding_lookup(embedding, labels)

            outputs = []
            with tf.variable_scope("GRU_generator"):
                for time_step in range(seq_size):
                    if time_step > 0: tf.get_variable_scope().reuse_variables()
                    inp = tf.nn.relu(tf.matmul(input_[:, 0, :], input_w) + input_b)

                    cell_output, state = cell(inp, state)
                    logits = tf.nn.softmax(tf.matmul(cell_output, softmax_w) + softmax_b)
                    labels = tf.expand_dims(tf.argmax(logits, 1), 1)
                    input_ = tf.nn.embedding_lookup(embedding, labels)
                    outputs.append(tf.expand_dims(logits, 1))

            output = tf.concat(outputs, 1)
        variables = [v for v in tf.global_variables() if 'generator_model' in v.name]

        return output, variables

    def build_discriminator(self, input_, is_train = False, reuse = False):
        vocab_size, hidden_size_disc, batch_size, seq_size,  keep_prob = \
            self.vocab_size, self.hidden_size_disc, \
            self.batch_size, self.seq_size, self.keep_prob

        with tf.variable_scope('discriminator_model', reuse = reuse):
            cell = tf.contrib.rnn.GRUCell(hidden_size_disc)
            if is_train:
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)

            if is_train:
                input_ = tf.nn.dropout(input_, keep_prob)

            state = cell.zero_state(batch_size, tf.float32)

            input_w = tf.get_variable(
                "input_w",
                [vocab_size, hidden_size_disc],
                initializer=tf.random_normal_initializer(0, stddev=1/np.sqrt(vocab_size))
            )
            input_b = tf.get_variable(
                "input_b",
                [hidden_size_disc],
                initializer=tf.constant_initializer(1e-4)
            )

            with tf.variable_scope("GRU_discriminator"):
                for time_step in range(seq_size):
                    if time_step > 0: tf.get_variable_scope().reuse_variables()
                    inp = tf.nn.relu(tf.matmul(input_[:, time_step, :], input_w) + input_b)
                    cell_output, state = cell(inp, state)

            out_w = tf.get_variable(
                "discriminator_output_w",
                [hidden_size_disc, 1],
                initializer=tf.random_normal_initializer(0, 1./np.sqrt(hidden_size_disc))
            )
            out_b = tf.get_variable(
                "discriminator_output_b",
                [1],
                initializer=tf.constant_initializer(1e-4)
            )

            output = tf.reduce_mean(tf.sigmoid(tf.matmul(cell_output, out_w) + out_b))

        variables = [v for v in tf.global_variables() if 'discriminator_model' in v.name]

        return output, variables

# Train

In [3]:
import os
import sys

import tensorflow as tf
import numpy as np

import utils
#from lstm_gan import LSTMGAN

DATASET_FILE = 'preprocessed.pickle'
VOCABULARY_FILE = 'index_to_word.pickle'

SENTENCE_SIZE = 30
VOCABULARY_SIZE = 10000

SENTENCE_START_TOKEN = "START"
SENTENCE_END_TOKEN = "END"

In [4]:
class Args(object):
    def __init__(self, disc_count=128, gen_count=10, data_path="data", 
                batch_size=512, hid_gen=512, hid_disc=512, dropout=0.2,
                grad_clip=0.1, noise_size=32, lr=1e-4, save_model=True,
                 load_model=False, gen_sent=False):
        self.disc_count = disc_count
        self.gen_count = gen_count
        self.data_path = data_path
        self.batch_size = batch_size
        self.hid_gen = hid_gen
        self.hid_disc = hid_disc
        self.dropout = dropout
        self.grad_clip = grad_clip
        self.noise_size = noise_size
        self.lr = lr
        self.save_model = save_model
        self.load_model = load_model
        self.gen_sent = gen_sent
            
args = Args(disc_count=32, gen_count=2, data_path="data", 
                batch_size=32, hid_gen=32, hid_disc=32, dropout=0.2,
                grad_clip=0.1, noise_size=32, lr=1e-4, save_model=True,
                 load_model=False, gen_sent=False)

In [5]:
dataset = utils.load_dataset(os.path.join(args.data_path, DATASET_FILE))
index2word, word2index = utils.load_dicts(os.path.join(args.data_path, VOCABULARY_FILE))

print("Use dataset with {} sentences".format(dataset.shape[0]))

Use dataset with 268833 sentences


In [ ]:
#args = utils.get_args()
dataset = utils.load_dataset(os.path.join(args.data_path, DATASET_FILE))
index2word, word2index = utils.load_dicts(os.path.join(args.data_path, VOCABULARY_FILE))

print("Use dataset with {} sentences".format(dataset.shape[0]))

batch_size = args.batch_size
noise_size = args.noise_size
with tf.Graph().as_default(), tf.Session() as session:   
    lstm_gan = LSTMGAN(
        SENTENCE_SIZE,
        VOCABULARY_SIZE,
        word2index[SENTENCE_START_TOKEN],
        hidden_size_gen = args.hid_gen,
        hidden_size_disc = args.hid_disc,
        input_noise_size = noise_size,
        batch_size = batch_size,
        dropout = args.dropout,
        lr = args.lr,
        grad_cap = args.grad_clip
    )

    session.run(tf.global_variables_initializer())

    if args.save_model or args.load_model:
        saver = tf.train.Saver()

    if args.load_model:
        try:
            saver.restore(session, utils.SAVER_FILE)
        except ValueError:
            print("Cant find model file")
            sys.exit(1)
            
    while True:
        offset = 0.
        for dataset_part in utils.iterate_over_dataset(dataset, batch_size*args.disc_count):
            print("Start train discriminator wih offset {}...".format(offset))
            for ind, batch in enumerate(utils.iterate_over_dataset(dataset_part, batch_size)):
                noise = np.random.random(size=(batch_size, noise_size))
                cost = lstm_gan.train_disc_on_batch(session, noise, batch)
                print("Processed {} sentences with train cost = {}".format((ind+1)*batch_size, cost))

            print("Start train generator...")
            for ind in range(args.gen_count):
                noise = np.random.random(size=(batch_size, noise_size))
                cost = lstm_gan.train_gen_on_batch(session, noise)
                if args.gen_sent:
                    sent = lstm_gan.generate_sent(session, np.random.random(size=(noise_size, )))
                    print(' '.join(index2word[i] for i in sent))
                print("Processed {} noise inputs with train cost {}".format((ind+1)*batch_size, cost))

            offset += batch_size*args.disc_count
            if args.save_model:
                #saver.save(session, utils.SAVER_FILE)
                saver.save(session, './model/model.ckpt')
                print("Model saved")

Use dataset with 268833 sentences
Start train discriminator wih offset 0.0...
Processed 32 sentences with train cost = 0.7511383295059204
Processed 64 sentences with train cost = 0.7510398030281067
Processed 96 sentences with train cost = 0.7508947849273682
Processed 128 sentences with train cost = 0.7507953643798828
Processed 160 sentences with train cost = 0.7506499290466309
Processed 192 sentences with train cost = 0.7505163550376892
Processed 224 sentences with train cost = 0.7503777742385864
Processed 256 sentences with train cost = 0.7502697706222534
Processed 288 sentences with train cost = 0.7501479983329773
Processed 320 sentences with train cost = 0.7500008344650269
Processed 352 sentences with train cost = 0.7498536705970764
Processed 384 sentences with train cost = 0.7497889399528503
Processed 416 sentences with train cost = 0.749627411365509
Processed 448 sentences with train cost = 0.7495055794715881
Processed 480 sentences with train cost = 0.7493674159049988
Processed 5

Processed 928 sentences with train cost = 0.7166145443916321
Processed 960 sentences with train cost = 0.7161576747894287
Processed 992 sentences with train cost = 0.7142102122306824
Processed 1024 sentences with train cost = 0.7143020629882812
Start train generator...
Processed 32 noise inputs with train cost 0.5306053757667542
Processed 64 noise inputs with train cost 0.5296124219894409
Model saved
Start train discriminator wih offset 4096.0...
Processed 32 sentences with train cost = 0.713300347328186
Processed 64 sentences with train cost = 0.712344765663147
Processed 96 sentences with train cost = 0.7112644910812378
Processed 128 sentences with train cost = 0.7108765840530396
Processed 160 sentences with train cost = 0.7094367146492004
Processed 192 sentences with train cost = 0.709808349609375
Processed 224 sentences with train cost = 0.7098826766014099
Processed 256 sentences with train cost = 0.7057808637619019
Processed 288 sentences with train cost = 0.7050157189369202
Proces

Processed 704 sentences with train cost = 0.13484936952590942
Processed 736 sentences with train cost = 0.12929612398147583
Processed 768 sentences with train cost = 0.12689030170440674
Processed 800 sentences with train cost = 0.13175415992736816
Processed 832 sentences with train cost = 0.1209908127784729
Processed 864 sentences with train cost = 0.12072145938873291
Processed 896 sentences with train cost = 0.1274624466896057
Processed 928 sentences with train cost = 0.11861836910247803
Processed 960 sentences with train cost = 0.11574983596801758
Processed 992 sentences with train cost = 0.11549389362335205
Processed 1024 sentences with train cost = 0.12351137399673462
Start train generator...
Processed 32 noise inputs with train cost 0.9261733293533325
Processed 64 noise inputs with train cost 0.9248388409614563
Model saved
Start train discriminator wih offset 8192.0...
Processed 32 sentences with train cost = 0.11051273345947266
Processed 64 sentences with train cost = 0.105387628

Processed 416 sentences with train cost = 0.03865694999694824
Processed 448 sentences with train cost = 0.03829914331436157
Processed 480 sentences with train cost = 0.038139939308166504
Processed 512 sentences with train cost = 0.03805917501449585
Processed 544 sentences with train cost = 0.03768497705459595
Processed 576 sentences with train cost = 0.03748750686645508
Processed 608 sentences with train cost = 0.03721219301223755
Processed 640 sentences with train cost = 0.03714549541473389
Processed 672 sentences with train cost = 0.036932170391082764
Processed 704 sentences with train cost = 0.03669935464859009
Processed 736 sentences with train cost = 0.03645700216293335
Processed 768 sentences with train cost = 0.03603774309158325
Processed 800 sentences with train cost = 0.03622168302536011
Processed 832 sentences with train cost = 0.03595954179763794
Processed 864 sentences with train cost = 0.036070168018341064
Processed 896 sentences with train cost = 0.03610110282897949
Proce

Processed 96 sentences with train cost = 0.024422407150268555
Processed 128 sentences with train cost = 0.024810612201690674
Processed 160 sentences with train cost = 0.023807883262634277
Processed 192 sentences with train cost = 0.024490654468536377
Processed 224 sentences with train cost = 0.023837029933929443
Processed 256 sentences with train cost = 0.023960769176483154
Processed 288 sentences with train cost = 0.023601949214935303
Processed 320 sentences with train cost = 0.02351146936416626
Processed 352 sentences with train cost = 0.023552238941192627
Processed 384 sentences with train cost = 0.02337491512298584
Processed 416 sentences with train cost = 0.02321648597717285
Processed 448 sentences with train cost = 0.02315080165863037
Processed 480 sentences with train cost = 0.023158371448516846
Processed 512 sentences with train cost = 0.023164451122283936
Processed 544 sentences with train cost = 0.022877991199493408
Processed 576 sentences with train cost = 0.0227277874946594

...

Processed 832 sentences with train cost = 0.004679560661315918
Processed 864 sentences with train cost = 0.004685521125793457
Processed 896 sentences with train cost = 0.0046784281730651855
Processed 928 sentences with train cost = 0.004666268825531006
Processed 960 sentences with train cost = 0.004654884338378906
Processed 992 sentences with train cost = 0.004647552967071533
Processed 1024 sentences with train cost = 0.004646718502044678
Start train generator...
Processed 32 noise inputs with train cost 0.9977906346321106
Processed 64 noise inputs with train cost 0.9977858662605286
Model saved
Start train discriminator wih offset 46080.0...
Processed 32 sentences with train cost = 0.004647195339202881
Processed 64 sentences with train cost = 0.004656076431274414
Processed 96 sentences with train cost = 0.004638552665710449
Processed 128 sentences with train cost = 0.004635751247406006
Processed 160 sentences with train cost = 0.00463557243347168
Processed 192 sentences with train cost

Processed 480 sentences with train cost = 0.004142105579376221
Processed 512 sentences with train cost = 0.004144072532653809
Processed 544 sentences with train cost = 0.004130542278289795
Processed 576 sentences with train cost = 0.004129469394683838
Processed 608 sentences with train cost = 0.004130840301513672
Processed 640 sentences with train cost = 0.004117786884307861
Processed 672 sentences with train cost = 0.004114806652069092
Processed 704 sentences with train cost = 0.004106879234313965
Processed 736 sentences with train cost = 0.004107356071472168
Processed 768 sentences with train cost = 0.004128217697143555
Processed 800 sentences with train cost = 0.004112303256988525
Processed 832 sentences with train cost = 0.004113554954528809
Processed 864 sentences with train cost = 0.004106104373931885
Processed 896 sentences with train cost = 0.004101932048797607
Processed 928 sentences with train cost = 0.004104793071746826
Processed 960 sentences with train cost = 0.00407958030

Processed 128 sentences with train cost = 0.0037015676498413086
Processed 160 sentences with train cost = 0.0037044286727905273
Processed 192 sentences with train cost = 0.003697693347930908
Processed 224 sentences with train cost = 0.0036894679069519043
Processed 256 sentences with train cost = 0.0036881566047668457
Processed 288 sentences with train cost = 0.0036861300468444824
Processed 320 sentences with train cost = 0.0036768317222595215
Processed 352 sentences with train cost = 0.0036728382110595703
Processed 384 sentences with train cost = 0.0036708712577819824
Processed 416 sentences with train cost = 0.0036696195602416992
Processed 448 sentences with train cost = 0.0036658644676208496
Processed 480 sentences with train cost = 0.003661036491394043
Processed 512 sentences with train cost = 0.0036590099334716797
Processed 544 sentences with train cost = 0.0036528706550598145
Processed 576 sentences with train cost = 0.0036516189575195312
Processed 608 sentences with train cost = 

Processed 896 sentences with train cost = 0.003318190574645996
Processed 928 sentences with train cost = 0.003312051296234131
Processed 960 sentences with train cost = 0.0033115744590759277
Processed 992 sentences with train cost = 0.0033093690872192383
Processed 1024 sentences with train cost = 0.0033025741577148438
Start train generator...
Processed 32 noise inputs with train cost 0.9984292984008789
Processed 64 noise inputs with train cost 0.998427152633667
Model saved
Start train discriminator wih offset 57344.0...
Processed 32 sentences with train cost = 0.0033034086227416992
Processed 64 sentences with train cost = 0.003312051296234131
Processed 96 sentences with train cost = 0.0032985806465148926
Processed 128 sentences with train cost = 0.0032962560653686523
Processed 160 sentences with train cost = 0.003294646739959717
Processed 192 sentences with train cost = 0.0032889246940612793
Processed 224 sentences with train cost = 0.003284931182861328
Processed 256 sentences with trai

Processed 544 sentences with train cost = 0.0030071139335632324
Processed 576 sentences with train cost = 0.003000319004058838
Processed 608 sentences with train cost = 0.0029959678649902344
Processed 640 sentences with train cost = 0.002989530563354492
Processed 672 sentences with train cost = 0.002989649772644043
Processed 704 sentences with train cost = 0.0029866695404052734
Processed 736 sentences with train cost = 0.0029816627502441406
Processed 768 sentences with train cost = 0.00298231840133667
Processed 800 sentences with train cost = 0.0029804110527038574
Processed 832 sentences with train cost = 0.002977430820465088
Processed 864 sentences with train cost = 0.0029758214950561523
Processed 896 sentences with train cost = 0.0029723048210144043
Processed 928 sentences with train cost = 0.0029689669609069824
Processed 960 sentences with train cost = 0.002967357635498047
Processed 992 sentences with train cost = 0.002966463565826416
Processed 1024 sentences with train cost = 0.002